1. data로드를 통한 데이터 프레임 완성
2. 한글에 맞게 형태소 분석기를 이용하여 단어 토큰화 작업후 tf-idf를 계산
3. 코사인 유사도 기반을 통한 영화 3종 추천

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

data=pd.read_csv("data2.csv")
data

,name,genre,content
0,올드보이,스릴러,술 좋아하고 떠들기 좋아하는 오.대.수. 본인의 이름풀이를 오늘만 대충 수습하며 살...
1,노트북,로맨스,"17살, ‘노아’는 밝고 순수한 ‘앨리’를 보고 첫눈에 반한다. 빠른 속도로 서로에..."
2,미션임파서블,액션,"도난당한 핵무기를 되찾아 오는 위험천만한 작전 중, 에단 헌트(톰 크루즈)는 임무를..."
3,니모를찾아서,애니메이션,“디즈니`픽사 최고의 애니메이션이 3D로 다시 찾아온다!” 호기심 가득한 아기 물고...
4,탑건,액션,해군 최신 전투기 F-14기를 모는 젊은 조종사 매버릭 대위(톰 크루즈)는 최고의 ...
5,친절한금자씨,스릴러,주변 사람들의 시선을 단번에 사로잡을 만큼 뛰어난 미모의 소유자인 금자(이영애)는 ...
6,반지의제왕,판타지,모든 힘을 지배할 악의 군주 ‘사우론’의 절대반지가 깨어나고 악의 세력이 세상을 지...
7,신세계,범죄,경찰청 수사 기획과 강과장(최민식)은 국내 최대 범죄 조직인 '골드문'이 기업형 조...
8,아저씨,NaN,불행한 사건으로 아내를 잃고 세상을 등진 채 전당포를 꾸려가며 외롭게 살아가는 전직...
9,해리포터,NaN,해리 포터(다니엘 래드클리프 분)는 위압적인 버논 숙부(리챠드 그리피스 분)와 냉담...


In [14]:
data['genre']=data['genre'].fillna('')

<class 'pandas.core.series.Series'>


In [60]:
from konlpy.tag import Okt
tk=Okt()
end_d=[]

for i in data['content']:
    tk_d=tk.morphs(i)
    l=[]
    합=""
    for j in tk_d:
        if len(j) > 1:
            if j != "..":
                l.append(j)
            elif j != "...":
                l.append(j)
    for k in l:
        l1=[]
        합 = 합 + k
    l1.append(합)
    end_d.append(l1)
end_d

idf_=pd.DataFrame(end_d,columns=['IDF'])
idf_
# end_d[0]
# tfidf = TfidfVectorizer(tk.morphs=)
# tfidf_t = tfidf.fit_transform(data['content'])
# tfidf_t




,IDF
0,좋아하고떠들기좋아하는본인이름풀이오늘대충수습하며살자라고이죽거리는남자아내어린딸아이가진지...
1,17노아밝고순수한앨리보고반한다빠른속도서로에게빠져드는그러나놓인장벽막혀이별하게된다24앨...
2,도난핵무기되찾아오는위험천만작전에단헌트크루즈임무포기하고목숨하는선택한다이제핵무기문명무너...
3,디즈니픽사최고애니메이션다시찾아온다호기심가득한아기물고기니모인간에게납치되자아들바보아빠말...
4,해군최신전투기14기를젊은조종사매버릭대위크루즈최고실력자부하는파일럿으로정예전투기조종사양...
5,주변사람시선단번사로잡을만큼뛰어난미모소유자이영애스무짓고감옥가게된다어린나이너무나아름다운...
6,모든지배악의군주사우론절대반지깨어나고악의세력세상지배해가며중간계혼란한다호빗프로도친구엘프...
7,경찰청수사기획과장최민식국내최대범죄조직골드기업조직으로세력점점확장되자신입경찰이자성이정재...
8,불행한사건으로아내잃고세상등진전당포꾸려가며외롭게살아가는전직특수요원태식찾아오는사람이라곤...
9,"해리포터다니엘래드클리프위압버논숙부챠드그리피스냉담한이모페투니피오),욕심많고버릇없는사촌..."


In [45]:
cos=cosine_similarity(tfidf_t,tfidf_t)
cos.shape

(18, 18)

In [24]:
data.index

RangeIndex(start=0, stop=18, step=1)

In [25]:
t_idx=dict(zip(data['name'],data.index))
t_idx

{'올드보이': 0,
 '노트북': 1,
 '미션임파서블': 2,
 '니모를찾아서': 3,
 '탑건': 4,
 '친절한금자씨': 5,
 '반지의제왕': 6,
 '신세계': 7,
 '아저씨': 8,
 '해리포터': 9,
 '장고': 10,
 '어바웃 타임': 11,
 '스타트렉': 12,
 '트랜스포머': 13,
 '지금만나러갑니다': 14,
 '007 노타임투다이': 15,
 '인터스텔라': 16,
 '스타워즈': 17}

In [26]:
def ck_s_t(t,cosine_sim=cos):
    idx=t_idx[t]
    c_sc=list(enumerate(cosine_sim[idx]))
    c_sc=sorted(c_sc,key=lambda x:x[1],reverse=True)#내림차순 정렬 크면클수록 유사도가 높다 최대치가 1이기 때문
    m_i=c_sc[1:4]#유사도 검사가 끝났을때 나 자신도 학습되기때문 첫번째 값을 빼야한다./인덱스는 영화제목
    m_idx=[i[0] for i in m_i]
    return data['name'].iloc[m_idx]

In [27]:
ck_s_t('장고')

16    인터스텔라
0      올드보이
7       신세계
Name: name, dtype: object

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

data=pd.read_csv("data2.csv")
data
data['genre']=data['genre'].fillna('')

from konlpy.tag import Okt
tk=Okt()
end_d=[]

for i in data['content']:
    tk_d=tk.morphs(i)
    l=[]
    합=""
    for j in tk_d:
        if len(j) > 1:
            if j != "..":
                l.append(j)
            elif j != "...":
                l.append(j)
    for k in l:
        l1=[]
        합 = 합+ " " + k
    l1.append(합)
    end_d.append(l1)

idf_=pd.DataFrame(end_d,columns=['IDF'])
idf_['IDF']

tfidf = TfidfVectorizer()
tfidf_t = tfidf.fit_transform(idf_['IDF'])


cos=cosine_similarity(tfidf_t,tfidf_t)
print(cos.shape)

print(data.index)

t_idx=dict(zip(data['name'],data.index))

def ck_s_t(t,cosine_sim=cos,k=0):
    idx=t_idx[t]
    c_sc=list(enumerate(cosine_sim[idx]))
    if k == 0:
        c_sc=sorted(c_sc,key=lambda x:x[1],reverse=True)
    else:
        c_sc=sorted(c_sc,key=lambda x:x[1])
    m_i=c_sc[1:4]
    m_idx=[i[0] for i in m_i]
    return data['name'].iloc[m_idx]
ck_s_t('올드보이')

(18, 18)
RangeIndex(start=0, stop=18, step=1)


5     친절한금자씨
11    어바웃 타임
8        아저씨
Name: name, dtype: object

In [62]:
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics import euclidean_distances
cos=cosine_similarity(tfidf_t,tfidf_t)#숫자가 높으면 유사도가 높다
c1=manhattan_distances(tfidf_t,tfidf_t)#숫자가 높으면 유사도가 낮다
c2=euclidean_distances(tfidf_t,tfidf_t)
len(cos),len(c1),len(c2)

(18, 18, 18)

In [73]:
ck_s_t('올드보이',k=1)

6     반지의제왕
9      해리포터
16    인터스텔라
Name: name, dtype: object